### Classification based on extracted features

In [5]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.models import Model
import numpy as np

In [6]:
model = VGG16(weights='imagenet', include_top=True)

In [7]:
features_model = Model(input=model.input, output=model.layers[-1].output)

/cyclope/nshvai/.nshvai/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("pr...)`
  """Entry point for launching an IPython kernel.


In [19]:
test_dir = '../data/training_folder/test'
val_dir = '../data/training_folder/val'
image_size = (224,224)

In [20]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import preprocess_input as preprocess_input_vgg


test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input_vgg) 
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input_vgg) 

test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=image_size,
        shuffle = False,
        class_mode='categorical',
        batch_size=1)

val_generator = val_datagen.flow_from_directory(
        val_dir,
        target_size=image_size,
        shuffle = False,
        class_mode='categorical',
        batch_size=1)

filenames = test_generator.filenames
nb_samples = len(filenames)

predict_test = model.predict_generator(test_generator,steps=nb_samples)
predict_val = model.predict_generator(val_generator,steps=nb_samples)

Found 340 images belonging to 17 classes.
Found 340 images belonging to 17 classes.


Now we'll use classical ML algorithm on extracted features

Let us use only the first two classes

In [24]:
features_test = predict_test[test_generator.classes <=1]
features_train = predict_val[val_generator.classes <=1]

y_test = test_generator.classes[test_generator.classes <=1]
y_train = val_generator.classes[val_generator.classes <=1]

In [30]:
from sklearn.linear_model import LogisticRegression

In [26]:
clf = LogisticRegression()

In [27]:
clf.fit(features_train, y_train)

/cyclope/nshvai/.nshvai/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [28]:
pred = clf.predict(features_test)

In [29]:
from sklearn.metrics import accuracy_score
accuracy_score(pred, y_test)

0.825